In [94]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results2/execute_sw.csv')


In [95]:
df.head()

,Instance,Algorithme,Seed,Score,Lambda,MaxEval,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,2307,5,100000,199.093
1,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,2426,9,100000,113.974
2,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,2632,5,100000,194.674
3,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,2525,9,100000,114.643
4,instances/atsp_rand_50_75.txt,sampled_walk_swap,52,2587,5,100000,193.871


In [96]:
neighbor_relations = ["swap","2opt"]
versus = ('swap', '2opt')
instances = df['Instance'].unique()
lambdas = df["Lambda"].unique()


In [97]:
def get_score(instance,neighbor_relation,seed,ld):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"sampled_walk_{neighbor_relation}") &
    (df['Seed'] == seed) & 
    (df["Lambda"] == ld)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,seed)
    

In [98]:
def gagne(instance,seed, vs,ld):
    #print("vs:",vs)
    s1 = get_score(instance,vs[0],seed,ld)
    s2 = get_score(instance,vs[1],seed,ld)
    return s1 < s2

In [99]:
get_score("instances/atsp_rand_50_75.txt","swap",50,9)

2426

In [100]:
get_score("instances/atsp_rand_50_75.txt","2opt",50,9)

2369

In [101]:
gagne("instances/atsp_rand_50_75.txt",50, ("swap","2opt"),9)

False

In [102]:
result_tuple={"instances" : instances, "swap_vs_2opt" : []}
for instance in instances:
    
    r=[0,0]
    for ld in lambdas:
        for seed in df["Seed"].unique():
            if gagne(instance,seed, versus,ld):
                r[0]+=1
            else:
                r[1]+=1
            
    result_tuple["swap_vs_2opt"].append((r[0],r[1]))
            
            #print(r)
                
result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_vs_2opt
0,instances/atsp_rand_50_75.txt,"(8, 12)"
1,instances/atsp_rand_50_100.txt,"(11, 9)"
2,instances/atsp_rand_60_50.txt,"(9, 11)"
3,instances/atsp_rand_60_100.txt,"(7, 13)"
4,instances/atsp_rand_70_55.txt,"(7, 13)"
5,instances/atsp_rand_70_100.txt,"(9, 11)"
6,instances/atsp_rand_80_70.txt,"(11, 9)"
7,instances/atsp_rand_80_100.txt,"(8, 12)"


In [103]:
df_group = df[["Algorithme","Score"]].groupby("Algorithme")
df_group.mean()

,Score
Algorithme,
sampled_walk_2opt,2833.88125
sampled_walk_swap,2864.96875
